In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.1 MB/s eta 0:00:00


In [2]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.7 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW
from sentence_transformers import SentenceTransformer
from transformers import get_cosine_schedule_with_warmup

In [9]:
def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/'
SEED = 42
reset_seeds(SEED)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [12]:
# 데이터 로드
train_ft = pd.read_csv(f'{DATA_PATH}train_ft_26440.csv')
test = pd.read_csv(f'{DATA_PATH}test.csv')
submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')

train_ft.shape

(26440, 3)

In [57]:
# 저장된 모델과 토크나이저 불러오기
from transformers import GPT2LMHeadModel

model_name = 'skt/kogpt2-base-v2'
finetune_model = 'kogpt2_trial2_date_2_13_epoch6_loss0.1751'

model = GPT2LMHeadModel.from_pretrained(f"{DATA_PATH}models/kogpt2_trial2_date_2_13_epoch6_loss0.1751")
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>')

In [14]:
embedding_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [70]:
@torch.no_grad()
def chatbot(df, model, tokenizer, device,
            max_length, temperature, top_k, top_p, repetition_penalty,
            do_sample, num_return_sequences):
    model.eval()
    model.to(device)
    preds = []
    for i in range(df.shape[0]):
        test_question = df['question'][i]
        test_question = "<q>" + test_question + "</s><a>"
        input_ids = tokenizer.encode(test_question, return_tensors='pt').to(device)
        result_ids = model.generate(input_ids=input_ids.to(device),
                                    max_length=max_length,
                                    temperature=temperature,
                                    top_k=top_k,
                                    top_p=top_p,
                                    repetition_penalty=repetition_penalty,
                                    do_sample=do_sample,
                                    num_return_sequences=num_return_sequences)
        q_len = len(test_question)
        result = tokenizer.decode(result_ids[0])
        preds.append(result[q_len+1:-4])
    return preds


In [73]:
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

In [74]:
train_ft

,category,question,answer
0,NaN,인테리어 디자인에서 미니멀리즘 스타일을 살리려면 어떤 디자인을 선택해야 하나요? 또...,"미니멀리즘 스타일은 간결하고 깔끔한 디자인을 추구합니다. 단순한 형태와 색상, 규칙..."
1,NaN,정배작업이 뭐야? 어떻게 공간을 미니멀리스트 스타일로 꾸밀 수 있을까요?,"정배작업은 벽지를 자르고, 벽지 안쪽 면에 도배풀을 도포한 후 벽면에 부착하는 일련..."
2,NaN,어떻게 아파트의 작은 발코니를 활용하여 아늑한 야외 공간을 만들 수 있을까요? 한편...,작은 발코니를 활용하여 아늑한 야외 공간을 만들기 위해서는 몇 가지 아이디어를 적용...
3,NaN,벽에 뚫린 구멍이 이상해 또 단열재의 종류 좀 알려줘,"벽이나 천장에 잘못 뚫린 구멍은 ""타공하자""라고 합니다. 타공하자가 발생하는 발생하..."
4,NaN,발수공사가 뭐야? 혹은 퍼티 마감의 종류는 뭐가 있어,"발수공사란 콘크리트 구체 및 모르타르, 벽돌 벽체 등 흡수성을 갖는 바탕재에 고분자..."
...,...,...,...
26435,NaN,실크벽지가 뭐야? 게다가 건조시간이 부족하면 도배지 꼬임이 발생할 수 있어?,"실크벽지란 종이벽지 표면에 PVC 코팅한 벽지를 의미합니다. 가격은 평당 6,000..."
26436,마감재,친환경벽지는 어떤 소재로 만들어지나요?,"친환경벽지는 인체에 유해한 환경호르몬 배출을 억제하고 실내공기를 정화하며, 향균 등..."
26437,마감재,온돌마루의 장점이 뭐야?,온돌마루의 장점은 강마루처럼 열전도율이 높아서 겨울철에 따뜻한 바닥을 경험할 수 있...
26438,NaN,"흡음재 중 다공질이 뭐야? 그뿐 아니라, 점토벽돌의 백화 현상이 뭐야?",흡음재 중 다공질은 무수히 많은 미세 구멍이나 틈이 있는 재료를 말합니다. 대표적인...


In [75]:
train_ft.iloc[:10, :]

,category,question,answer
0,NaN,인테리어 디자인에서 미니멀리즘 스타일을 살리려면 어떤 디자인을 선택해야 하나요? 또...,"미니멀리즘 스타일은 간결하고 깔끔한 디자인을 추구합니다. 단순한 형태와 색상, 규칙..."
1,NaN,정배작업이 뭐야? 어떻게 공간을 미니멀리스트 스타일로 꾸밀 수 있을까요?,"정배작업은 벽지를 자르고, 벽지 안쪽 면에 도배풀을 도포한 후 벽면에 부착하는 일련..."
2,NaN,어떻게 아파트의 작은 발코니를 활용하여 아늑한 야외 공간을 만들 수 있을까요? 한편...,작은 발코니를 활용하여 아늑한 야외 공간을 만들기 위해서는 몇 가지 아이디어를 적용...
3,NaN,벽에 뚫린 구멍이 이상해 또 단열재의 종류 좀 알려줘,"벽이나 천장에 잘못 뚫린 구멍은 ""타공하자""라고 합니다. 타공하자가 발생하는 발생하..."
4,NaN,발수공사가 뭐야? 혹은 퍼티 마감의 종류는 뭐가 있어,"발수공사란 콘크리트 구체 및 모르타르, 벽돌 벽체 등 흡수성을 갖는 바탕재에 고분자..."
5,인테리어,현대적인 거실을 꾸밀 때 주의해야 할 점은 무엇인가요?,"현대적인 거실을 만들려면 미니멀 디자인, 현대적 가구, 그리고 간결한 컬러 팔레트를..."
6,NaN,세라믹사이딩의 단점이 뭐야? 그뿐 아니라 천장에 물이 새는 경우 어떻게 조치해야 하나요?,세라믹사이딩의 단점은 대부분 일본에서 수입해오기 때문에 자재비가 상대적으로 비싸다는...
7,NaN,"남은 도배풀은 어디에 버려? 혹은, 실내 조명을 선택할 때 어떤 종류의 조명을 고려...",남은 도배풀은 일반 종량제 봉투에 버리셔야 합니다. 반드시 지역의 쓰레기 처리 규정...
8,NaN,워쉬프라이머가 뭐야? 게다가 벽지에서 크랙이 발생한 경우 어떻게 처리해야 하나요?,워쉬프라이머는 합성수지를 전색제로 하여 안료와 인산을 첨가한 것으로 주로 뿜칠로 도...
9,마감재,플로킹 벽지란 무엇인가요?,"플로킹 벽지는 벨벳 소재를 사용하여 입체감을 살린 벽지로, 고급스러운 분위기를 연출..."


In [76]:
def gpt2_objective(trial):
    params = {
        'max_length': trial.suggest_int('max_length', 200, 400, step=50),
        'temperature': trial.suggest_float('temperature', 0.50, 0.95, step=0.01),
        'top_k': trial.suggest_int('top_k', 1, 90),
        'top_p': trial.suggest_float('top_p', 0.1, 0.9, step=0.1),
        'repetition_penalty': trial.suggest_float('repetition_penalty', 0.5, 1.5, step=0.1),
        'do_sample': True,
        'num_return_sequences': 1,
        # 'num_beams' : trial.suggest_int('num_beams',0,10),
        # 'no_repeat_ngram_size' : 1
    }
    preds = chatbot(train_ft.iloc[:10, :], model, tokenizer, device, **params)
    valids = train_ft.iloc[:10, :]['question']
    sample_scores = []
    for pred, valid in zip(preds, valids):
        pred_embed = embedding_model.encode(pred)
        valid_embed = embedding_model.encode(valid)

        sample_score = cosine_similarity(valid_embed, pred_embed)
        sample_score = max(sample_score, 0)
        sample_scores.append(sample_score)

    return np.mean(sample_scores)

In [77]:
import optuna

sampler = optuna.samplers.TPESampler(seed=SEED)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(gpt2_objective, n_trials=50, show_progress_bar = True)

[I 2024-02-21 09:36:20,520] A new study created in memory with name: no-name-318a1f53-f509-4607-910a-fe320fde70fc


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2024-02-21 09:36:36,220] Trial 0 finished with value: 0.6239161491394043 and parameters: {'max_length': 250, 'temperature': 0.9299999999999999, 'top_k': 66, 'top_p': 0.6, 'repetition_penalty': 0.6}. Best is trial 0 with value: 0.6239161491394043.
[I 2024-02-21 09:36:50,157] Trial 1 finished with value: 0.435223251581192 and parameters: {'max_length': 200, 'temperature': 0.52, 'top_k': 78, 'top_p': 0.6, 'repetition_penalty': 1.2000000000000002}. Best is trial 0 with value: 0.6239161491394043.
[I 2024-02-21 09:37:03,875] Trial 2 finished with value: 0.5343533158302307 and parameters: {'max_length': 200, 'temperature': 0.94, 'top_k': 75, 'top_p': 0.2, 'repetition_penalty': 0.7}. Best is trial 0 with value: 0.6239161491394043.
[I 2024-02-21 09:37:16,013] Trial 3 finished with value: 0.4652879238128662 and parameters: {'max_length': 200, 'temperature': 0.63, 'top_k': 48, 'top_p': 0.4, 'repetition_penalty': 0.8}. Best is trial 0 with value: 0.6239161491394043.
[I 2024-02-21 09:37:31,083] 

KeyboardInterrupt: 

In [38]:
import optuna

sampler = optuna.samplers.TPESampler(seed=SEED)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(gpt2_objective, n_trials=50, show_progress_bar = True)

[I 2024-02-21 07:14:41,506] A new study created in memory with name: no-name-69d9661e-50bd-467b-b31d-e8e51847f9ec


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2024-02-21 07:16:50,433] Trial 0 finished with value: 0.6447216868400574 and parameters: {'max_length': 300, 'temperature': 0.91, 'top_k': 66, 'top_p': 0.6, 'repetition_penalty': 0.6}. Best is trial 0 with value: 0.6447216868400574.
[I 2024-02-21 07:18:42,356] Trial 1 finished with value: 0.46467599272727966 and parameters: {'max_length': 250, 'temperature': 0.14, 'top_k': 78, 'top_p': 0.6, 'repetition_penalty': 1.2000000000000002}. Best is trial 0 with value: 0.6447216868400574.
[I 2024-02-21 07:20:43,618] Trial 2 finished with value: 0.5443771481513977 and parameters: {'max_length': 200, 'temperature': 0.93, 'top_k': 75, 'top_p': 0.2, 'repetition_penalty': 0.7}. Best is trial 0 with value: 0.6447216868400574.
[I 2024-02-21 07:22:49,998] Trial 3 finished with value: 0.4934445917606354 and parameters: {'max_length': 250, 'temperature': 0.36, 'top_k': 48, 'top_p': 0.4, 'repetition_penalty': 0.8}. Best is trial 0 with value: 0.6447216868400574.
[I 2024-02-21 07:24:53,884] Trial 4 fini

In [40]:
study.best_params

{'max_length': 500,
 'temperature': 0.64,
 'top_k': 29,
 'top_p': 0.7000000000000001,
 'repetition_penalty': 0.5}

In [ ]:
# {'max_length': 500,
#  'temperature': 0.64,
#  'top_k': 29,
#  'top_p': 0.7000000000000001,
#  'repetition_penalty': 0.5}

In [83]:
@torch.no_grad()
def chatbot(df, model, tokenizer, device,
            max_length, temperature, top_k, top_p, repetition_penalty,
            do_sample, num_return_sequences):
    model.eval()
    model.to(device)
    preds = []
    for i in tqdm(range(df.shape[0])):
        test_question = df['질문'][i]
        test_question = "<q>" + test_question + "</s><a>"
        input_ids = tokenizer.encode(test_question, return_tensors='pt').to(device)
        result_ids = model.generate(input_ids=input_ids.to(device),
                                    max_length=max_length,
                                    temperature=temperature,
                                    top_k=top_k,
                                    top_p=top_p,
                                    repetition_penalty=repetition_penalty,
                                    do_sample=do_sample,
                                    num_return_sequences=num_return_sequences)
        q_len = len(test_question)
        result = tokenizer.decode(result_ids[0])
        preds.append(result[q_len+1:-4])
    return preds

In [80]:
best_params = {'max_length': 350, 'temperature': 0.59, 'top_k': 47, 'top_p': 0.6, 'repetition_penalty': 0.5}

In [82]:
test

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
...,...,...
125,TEST_125,분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요? 그리고 아파트 도배 평수를...
126,TEST_126,"압출법 보온판의 가장 큰 장점은 무엇인가요?"""
127,TEST_127,평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...
128,TEST_128,석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 ...


In [84]:
best_params = study.best_params
best_params = {'max_length': 350, 'temperature': 0.59, 'top_k': 47, 'top_p': 0.6, 'repetition_penalty': 0.5}

preds = chatbot(test, model, tokenizer, device, do_sample=True, num_return_sequences=1, **best_params)
preds

  0%|          | 0/130 [00:00<?, ?it/s]

['방청 페인트의 종류에는 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방 페인트의 광방청 페인트의 광방 페인트의 광방청 페인트의 광방 페인트의 광방청 페인트의 광방 페인트의 광방청 페인트의 광방 페인트의 광방청 페인트의 광방 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방 페인트의 광방청 페인트의 광방 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광방청 페인트의 광',
 '도배지에 녹은 녹은 녹오염은 녹오염>도배지에 녹오염은 발생하는 주된 원인과 해결 방법은 무엇인가요?',
 '큐블럭의 단점을 알려주세요.',
 '철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철골철',
 '도배지의 완전한 건조를 위해 몇 주 기다려야 합니다.',
 '철근철골콘크리트 구조가 적용된 건물의 장점은철근콘크리트 구조가 적용된 건물의 장점은 철근콘크리트 구조가 적용된 철골콘크리트 구조가 철근콘크리트

In [51]:
# Embedding Vector 추출

pred_vector = embedding_model.encode(preds)
pred_vector.shape

(130, 512)

In [52]:
submission.iloc[:,1:] = pred_vector
submission

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.032735,-0.000309,0.023209,-0.002054,-0.060259,0.018331,0.014933,-0.005695,0.060672,...,0.018889,-0.015689,0.015430,-0.038796,0.002761,-0.044169,0.047314,0.036126,-0.002156,0.049845
1,TEST_001,-0.025622,-0.001084,0.013519,0.008752,-0.012744,-0.025202,0.000539,-0.075895,0.008344,...,-0.042742,0.006871,0.010605,-0.078468,0.034081,0.005302,-0.010239,-0.035141,0.037727,0.002460
2,TEST_002,0.029276,-0.053586,-0.047715,0.044166,-0.069295,-0.000630,-0.044126,-0.039716,0.031278,...,0.008570,-0.010826,0.052062,-0.036087,0.024702,-0.016457,0.011927,0.020443,-0.038031,0.000387
3,TEST_003,0.024923,-0.003076,-0.034033,-0.018650,-0.045935,0.008263,0.000726,-0.054364,-0.002241,...,0.015663,-0.016918,0.002475,0.025275,-0.051499,-0.064156,0.005965,-0.020918,0.016907,-0.020308
4,TEST_004,0.002056,-0.022960,0.028003,-0.012492,-0.060599,0.000852,0.079402,0.032745,0.027971,...,0.018947,-0.008115,0.030324,0.005614,-0.030539,0.030630,0.010665,0.089071,-0.033211,-0.012364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,TEST_125,0.047948,-0.058072,0.016637,0.003617,0.121776,-0.015789,0.060263,-0.001190,0.049005,...,-0.012753,0.031762,0.015772,-0.033899,0.018713,0.034063,-0.011687,-0.054921,-0.017861,0.026031
126,TEST_126,0.043185,-0.072568,-0.040507,0.026999,0.107343,0.033447,0.024943,0.011051,0.030722,...,-0.045322,-0.067330,0.024210,-0.034120,0.041013,0.044163,-0.004861,-0.032927,0.027454,0.034489
127,TEST_127,0.039730,0.024889,-0.046164,0.040154,0.011180,0.011516,-0.025179,0.031703,-0.032315,...,-0.036823,-0.087545,0.021646,-0.033191,-0.037525,0.033769,0.041818,-0.014791,0.028229,-0.008548
128,TEST_128,0.034636,-0.021116,-0.085064,0.007311,0.084639,-0.035019,0.005997,0.000921,0.003620,...,-0.058310,0.045952,-0.033606,0.001924,0.031804,0.040856,-0.027504,-0.029995,0.027001,-0.013470


In [53]:
submission.to_csv(f'kogpt_optuna_0221.csv', index=False)